In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
%%capture
%run "37. Nuevos CT Filtrados.ipynb"

In [3]:
Columnas_Cambios = ['Cambio_Tiempo_Filt_Pro_Izq', 
                    'Cambio_Tiempo_Filt_Con_Izq', 
                    'Cambio_Tiempo_Filt_Pro_Der', 
                    'Cambio_Tiempo_Filt_Con_Der'
                    ]

In [ ]:
for Nombre_df, df in dfs_Finales.items():

   Columnas = ['ID', 'Categoria_PASO_2023'] + Columnas_Cambios

   df_Copia = df[Columnas]

   # Filtrar filas con categorías válidas y sin valores nulos en las columnas de interés.
   Categorias_Validas = [
       'Left_Wing', 
       'Progressivism', 
       'Centre',
       'Moderate_Right_A', 
       'Moderate_Right_B',
       'Right_Wing_Libertarian'
   ]

   df_Copia = df_Copia[
       df_Copia['Categoria_PASO_2023'].isin(Categorias_Validas)
   ].dropna(subset=Columnas_Cambios)

   from scipy.stats import kruskal
   import pandas as pd

   # Crear diccionario para almacenar resultados.
   Resultados_Kruskal = {}

   # Agrupar por categoría para cada columna y aplicar Kruskal-Wallis.
   for Nombre_Columna in Columnas_Cambios:
       try:
           Grupos = [
               df_Copia[
                   df_Copia['Categoria_PASO_2023'] == Categoria
               ][Nombre_Columna]
               for Categoria in Categorias_Validas
           ]
           
           # Verificar si hay variabilidad en los datos.
           Datos_Combinados = pd.concat(Grupos, ignore_index=True)
           
           if Datos_Combinados.nunique() <= 1:
               # Todos los valores son idénticos - omitir esta columna.
               continue
           else:
               # Ejecutar test de Kruskal-Wallis.
               Estadistico, P_Valor = kruskal(*Grupos)
               Resultados_Kruskal[Nombre_Columna] = {
                   'Estadístico': Estadistico,
                   'Valor_p': P_Valor
               }
               
       except ValueError as Error:
           if 'All numbers are identical' in str(Error):
               # Omitir columnas con valores idénticos.
               continue
           else:
               # Re-lanzar otros errores de ValueError.
               raise Error
   
   # Convertir los resultados a DataFrame.
   Tabla_Resultados_Kruskal = pd.DataFrame.from_dict(
       Resultados_Kruskal, orient='index'
   ).reset_index().rename(columns={
       'index': 'Variable',
       'Estadístico': 'Estadistico_Kruskal',
       'Valor_p': 'Valor_p'
   })

   # Ordenar por p-valor ascendente.
   Tabla_Resultados_Kruskal = Tabla_Resultados_Kruskal.sort_values(
       by='Valor_p'
   ).reset_index(drop=True)

   # Filtrar variables significativas (p-valor < 0.05).
   Variables_Significativas = Tabla_Resultados_Kruskal[
       Tabla_Resultados_Kruskal['Valor_p'] < 0.05
   ]['Variable'].tolist()
   
   Todas_Variables_Significativas[
       'Cambios_Tiempo_Filtrados_Significativos'
   ][Nombre_df] = Variables_Significativas

   # Mostrar tabla.
   print(Tabla_Resultados_Kruskal)